In [1]:
#!/usr/bin/env python

import os
import json
import pandas as pd
import numpy as np
from datetime import datetime

# ============================================================
# CONFIG
# ============================================================

UNIVERSE_FILE   = "./12-tradable_sp500_universe/12-tradable_sp500_universe.parquet"
SPY_REGIME_FILE = "./8-SPY_200DMA_market_regime/8-SPY_200DMA_regime.parquet"
ATR20_DIR       = "./4-ATR20_adjusted_All_Prices"

OUTPUT_ROOT     = "./26-simple_parameter_sweep"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

START_TRADING       = pd.Timestamp("1999-01-01")
INITIAL_CAPITAL     = 365_000
EXEC_DELAY_DAYS     = 1      # <<< T+1 EXECUTION

# sweep parameters
RISK_VALUES        = [0.0005, 0.0010, 0.0020]
TOP_PCT_VALUES     = [0.75, 0.85, 0.90, 0.92]
REBALANCE_DAYS     = ["Monday", "Wednesday", "Friday"]

# ============================================================
# FAST PRICE LOOKUP
# ============================================================

def fast_price_lookup(px_array, date_val):
    date_val = np.datetime64(date_val, "ns")
    dates = px_array["date"]
    idx = np.searchsorted(dates, date_val, side="right") - 1
    if idx < 0:
        return np.nan
    return px_array["px"][idx]

# ============================================================
# LOAD DATA
# ============================================================

print("Loading universe & SPY data...")

df = pd.read_parquet(UNIVERSE_FILE)
df["date"] = pd.to_datetime(df["date"])
df["slope_adj"] = pd.to_numeric(df["slope_adj"], errors="coerce")
df["close_adj"] = pd.to_numeric(df["close_adj"], errors="coerce")

spy = pd.read_parquet(SPY_REGIME_FILE).reset_index().rename(columns={"Date": "date"})
spy["date"] = pd.to_datetime(spy["date"])

df = df.merge(
    spy[["date", "spy_close", "spy_ma200", "market_regime"]],
    on="date",
    how="left"
)

# ATR20
atr20_map = {}
for f in os.listdir(ATR20_DIR):
    if f.endswith(".parquet"):
        t = f.replace(".parquet", "")
        tmp = pd.read_parquet(os.path.join(ATR20_DIR, f))
        if "atr20" in tmp.columns:
            tmp["date"] = pd.to_datetime(tmp["date"])
            atr20_map[t] = tmp[["date", "atr20"]]

rows = []
for t, sub in df.groupby("ticker", sort=False):
    if t in atr20_map:
        rows.append(sub.merge(atr20_map[t], on="date", how="left"))
    else:
        sub = sub.copy()
        sub["atr20"] = np.nan
        rows.append(sub)

df = pd.concat(rows, ignore_index=True)

df_by_date = {d: sub for d, sub in df.groupby("date")}
dates = sorted(df_by_date.keys())

px_by_ticker = {}
for t, sub in df.groupby("ticker", sort=False):
    sub = sub.sort_values("date")
    arr = np.zeros(len(sub), dtype=[("date", "datetime64[ns]"), ("px", "float64")])
    arr["date"] = sub["date"].values
    arr["px"] = sub["close_adj"].values
    px_by_ticker[t] = arr

# ============================================================
# BACKTEST ENGINE (T+1 EXECUTION)
# ============================================================

def run_backtest(RISK_PER_TRADE, TOP_PERCENTILE, REBALANCE_DAY):

    cash = INITIAL_CAPITAL
    positions = {}   # ticker -> shares
    equity_curve = []
    trades = []

    pending_orders = []   # queued signals waiting for execution

    def is_rebalance_day(d):
        return d.day_name() == REBALANCE_DAY

    for i, date in enumerate(dates):

        if date < START_TRADING:
            continue

        # ============================
        # EXECUTE PENDING ORDERS
        # ============================
        exec_orders = [o for o in pending_orders if o["exec_date"] == date]
        pending_orders = [o for o in pending_orders if o["exec_date"] != date]

        for o in exec_orders:
            t = o["ticker"]
            px = fast_price_lookup(px_by_ticker[t], date)
            if np.isnan(px):
                continue

            if o["side"] == "BUY":
                cost = o["shares"] * px
                if cost <= cash:
                    cash -= cost
                    positions[t] = positions.get(t, 0) + o["shares"]

            else:  # SELL
                shares = min(positions.get(t, 0), o["shares"])
                cash += shares * px
                positions[t] -= shares
                if positions[t] <= 0:
                    positions.pop(t, None)

            trades.append({
                "signal_date": o["signal_date"],
                "exec_date": date,
                "ticker": t,
                "side": o["side"],
                "shares": o["shares"],
                "price": px,
                "value": o["shares"] * px,
                "exec_delay_days": EXEC_DELAY_DAYS,
            })

        # ============================
        # SIGNAL GENERATION
        # ============================
        if is_rebalance_day(date):

            day = df_by_date.get(date)
            if day is None:
                continue

            regime_bull = day["market_regime"].iloc[0] == 1

            rankable = day[
                day["slope_adj"].notna()
                & day["in_sp500"]
                & day["no_big_jump_90"]
            ].sort_values("slope_adj", ascending=False)

            if len(rankable) >= 5:
                cutoff = rankable["slope_adj"].quantile(TOP_PERCENTILE)
                top = rankable[rankable["slope_adj"] >= cutoff]
            else:
                top = pd.DataFrame()

            exec_idx = i + EXEC_DELAY_DAYS
            if exec_idx >= len(dates):
                continue
            exec_date = dates[exec_idx]

            # SELL signals
            for t in list(positions.keys()):
                if t not in top["ticker"].values:
                    pending_orders.append({
                        "signal_date": date,
                        "exec_date": exec_date,
                        "ticker": t,
                        "side": "SELL",
                        "shares": positions[t],
                    })

            # BUY signals
            if regime_bull and not top.empty:
                equity = cash + sum(
                    fast_price_lookup(px_by_ticker[t], date) * sh
                    for t, sh in positions.items()
                )

                for _, r in top.iterrows():
                    t = r["ticker"]
                    if t in positions:
                        continue

                    atr = r["atr20"]
                    if pd.isna(atr) or atr <= 0:
                        continue

                    shares = int((equity * RISK_PER_TRADE) / atr)
                    if shares > 0:
                        pending_orders.append({
                            "signal_date": date,
                            "exec_date": exec_date,
                            "ticker": t,
                            "side": "BUY",
                            "shares": shares,
                        })

        # ============================
        # MARK TO MARKET
        # ============================
        equity = cash + sum(
            fast_price_lookup(px_by_ticker[t], date) * sh
            for t, sh in positions.items()
        )

        equity_curve.append({
            "date": date,
            "portfolio_value": equity,
            "cash": cash,
            "num_positions": len(positions),
        })

    return pd.DataFrame(trades), pd.DataFrame(equity_curve)

# ============================================================
# PARAMETER SWEEP
# ============================================================

print("\n=== STARTING SIMPLE PARAMETER SWEEP (T+1) ===\n")

for RP in RISK_VALUES:
    for TP in TOP_PCT_VALUES:
        for RD in REBALANCE_DAYS:

            label = f"RP{RP}_TP{TP}_RD{RD}"
            out_dir = os.path.join(OUTPUT_ROOT, label.replace(".", "p"))
            os.makedirs(out_dir, exist_ok=True)

            print(f"Running {label} ...")

            trades, equity = run_backtest(RP, TP, RD)

            trades.to_parquet(os.path.join(out_dir, "trades.parquet"), index=False)
            equity.to_parquet(os.path.join(out_dir, "equity.parquet"), index=False)

            summary = {
                "risk_per_trade": RP,
                "top_percentile": TP,
                "rebalance_day": RD,
                "exec_delay_days": EXEC_DELAY_DAYS,
                "final_value": float(equity["portfolio_value"].iloc[-1]),
                "num_trades": len(trades),
            }

            with open(os.path.join(out_dir, "summary.json"), "w") as f:
                json.dump(summary, f, indent=4)

            print(f"Completed {label}: {summary['final_value']:,.2f}")

print("\n=== SWEEP COMPLETE ===")


Loading universe & SPY data...

=== STARTING SIMPLE PARAMETER SWEEP (T+1) ===

Running RP0.0005_TP0.75_RDMonday ...
Completed RP0.0005_TP0.75_RDMonday: 4,217,277.49
Running RP0.0005_TP0.75_RDWednesday ...
Completed RP0.0005_TP0.75_RDWednesday: 4,355,417.61
Running RP0.0005_TP0.75_RDFriday ...
Completed RP0.0005_TP0.75_RDFriday: 3,809,083.25
Running RP0.0005_TP0.85_RDMonday ...
Completed RP0.0005_TP0.85_RDMonday: 3,522,677.25
Running RP0.0005_TP0.85_RDWednesday ...
Completed RP0.0005_TP0.85_RDWednesday: 4,110,954.01
Running RP0.0005_TP0.85_RDFriday ...
Completed RP0.0005_TP0.85_RDFriday: 3,167,413.96
Running RP0.0005_TP0.9_RDMonday ...
Completed RP0.0005_TP0.9_RDMonday: 2,904,900.37
Running RP0.0005_TP0.9_RDWednesday ...
Completed RP0.0005_TP0.9_RDWednesday: 3,016,058.18
Running RP0.0005_TP0.9_RDFriday ...
Completed RP0.0005_TP0.9_RDFriday: 2,678,788.36
Running RP0.0005_TP0.92_RDMonday ...
Completed RP0.0005_TP0.92_RDMonday: 2,365,935.42
Running RP0.0005_TP0.92_RDWednesday ...
Completed